# Import packages; read in files

In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import os
import seaborn as sns

In [2]:
# Chloe's directory
# Set file path
#os.chdir("/Users/chloemaine/Documents/Chloe/BGSE/masters_project/raw_data")

In [2]:
# Read in files


#'ccs_multi_dx.csv' is downloaded from https://www.hcup-us.ahrq.gov/toolssoftware/ccs/Multi_Level_CCS_2015.zip
#It's Clinical Classification Software (CCS) that categorises ICD-9 codes developed by the Agency for Healthcare Research and Quality (AHRQ)
icds = pd.read_csv('../input_data/ccs_multi_dx.csv') # Diagnosis codes

#'diagnosis_icd.csv' is the DIAGNOSIS_ICD table from MIMIC
diagnoses = pd.read_csv('../input_data/diagnosis_icd.csv') 

#'patient_details.csv' is created from a combination of multiple tables from MIMIC LCP
#The process I followed is https://github.com/MIT-LCP/mimic-code/blob/master/concepts/demographics/icustay-detail.sql
patient_details = pd.read_csv('../input_data/patient_details.csv')

#icustays_collapsed table from bgse-dsc (processed icustays with backshifted dates)
intime_collapsed = pd.read_csv('../input_data/INTIME_COLLAPSED.csv')
intime_collapsed = intime_collapsed[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'INTIME_COLLAPSED', 'OUTTIME_COLLAPSED']]

#'icustays.csv' is the ICUSTAYS table from MIMIC
icus = pd.read_csv('../input_data/icustays.csv').set_index('ICUSTAY_ID')

In [3]:
patient_details.head()

,subject_id,hadm_id,icustay_id,gender,dod,admittime,dischtime,los_hospital,age,ethnicity,ethnicity_grouped,admission_type,hospital_expire_flag,hospstay_seq,first_hosp_stay,intime,outtime,los_icu,icustay_seq,first_icu_stay
0,2,163353,243653,M,NaN,2138-07-17 19:04:00,2138-07-21 15:48:00,4,0,ASIAN,asian,NEWBORN,0,1,True,2138-07-17 21:20:07,2138-07-17 23:32:21,0.0,1,True
1,3,145834,211552,M,2102-06-14 00:00:00,2101-10-20 19:08:00,2101-10-31 13:58:00,11,76,WHITE,white,EMERGENCY,0,1,True,2101-10-20 19:10:11,2101-10-26 20:43:09,6.0,1,True
2,4,185777,294638,F,NaN,2191-03-16 00:28:00,2191-03-23 18:41:00,7,48,WHITE,white,EMERGENCY,0,1,True,2191-03-16 00:29:31,2191-03-17 16:46:31,1.0,1,True
3,5,178980,214757,M,NaN,2103-02-02 04:31:00,2103-02-04 12:15:00,2,0,ASIAN,asian,NEWBORN,0,1,True,2103-02-02 06:04:24,2103-02-02 08:06:00,0.0,1,True
4,6,107064,228232,F,NaN,2175-05-30 07:15:00,2175-06-15 16:00:00,16,66,WHITE,white,ELECTIVE,0,1,True,2175-05-30 21:30:54,2175-06-03 13:39:54,4.0,1,True


In [4]:
patient_details.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'gender', 'dod', 'admittime',
       'dischtime', 'los_hospital', 'age', 'ethnicity', 'ethnicity_grouped',
       'admission_type', 'hospital_expire_flag', 'hospstay_seq',
       'first_hosp_stay', 'intime', 'outtime', 'los_icu', 'icustay_seq',
       'first_icu_stay'],
      dtype='object')

In [5]:
patient_details.shape

(61532, 20)

In [6]:
intime_collapsed.shape

(61532, 5)

# Functions 

In [7]:

def extract_category(ccs_group1):
    return ccs_g1_categories[ccs_group1]

# Dictionaries

In [8]:
#The Clinical Classification Software (CCS) categorizes ICD-9 coded diagnoses into clinically meaningful groups. 
#The categorization was developed by the Agency for Healthcare Research and Quality (AHRQ). 
#the AHRQ website: https://www.hcup-us.ahrq.gov/tools_software.jsp


ccs_g1_categories = {
   'Diseases of the digestive system': 'digestive',
   'Diseases of the circulatory system': 'circulatory',
   'Diseases of the genitourinary system': 'genitourinary',
   'Infectious and parasitic diseases': 'infectious|parasitic',
   'Endocrine; nutritional; and metabolic diseases and immunity disorders': 'metabolic|immunity',
   'Diseases of the respiratory system': 'respiratory',
   'Injury and poisoning': 'injury|poisoning',
   'Symptoms; signs; and ill-defined conditions and factors influencing health status': 'ill-defined',
   'Diseases of the blood and blood-forming organs': 'blood',
   'Certain conditions originating in the perinatal period': 'perinatal',
   'Residual codes; unclassified; all E codes [259. and 260.]': 'unclassified',
   'Neoplasms': 'neoplasms',
   'Diseases of the nervous system and sense organs': 'nervous',
   'Mental Illness': 'mental',
   'Congenital anomalies': 'congenital',
   'Diseases of the musculoskeletal system and connective tissue': 'musculoskeletal',
   'Diseases of the skin and subcutaneous tissue': 'skin',
   'Complications of pregnancy; childbirth; and the puerperium': 'pregnancy complications',
   'Mental illness': 'mental'
}

# Join diagnoses to icds and extract sepsis ids

In [9]:
print('ICDs')
print(icds.shape)
icds.iloc[1:5, :]

ICDs
(15072, 9)


,icd9_code,ccs_level1,ccs_group1,ccs_level2,ccs_group2,ccs_level3,ccs_group3,ccs_level4,ccs_group4
1,01001,1,Infectious and parasitic diseases,1.1,Bacterial infection,1.1.1,Tuberculosis [1.],NaN,NaN
2,01002,1,Infectious and parasitic diseases,1.1,Bacterial infection,1.1.1,Tuberculosis [1.],NaN,NaN
3,01003,1,Infectious and parasitic diseases,1.1,Bacterial infection,1.1.1,Tuberculosis [1.],NaN,NaN
4,01004,1,Infectious and parasitic diseases,1.1,Bacterial infection,1.1.1,Tuberculosis [1.],NaN,NaN


In [10]:
diagnoses['ICD9_CODE'] = diagnoses['ICD9_CODE'].astype('str').str.strip() # removing any whitespaces
diagnoses = diagnoses.dropna(subset=['ICD9_CODE']) # look for missing values in 'ICD9_CODE'

icds['icd9_code'] = icds['icd9_code'].astype('str').str.strip()

print('Pre-merge:')
print('Diagnoses shape: ', diagnoses.shape)
print('icds shape: ', icds.shape)
joined = diagnoses.join(icds.set_index('icd9_code'), how='left', on='ICD9_CODE')
print('Post-merge:')
print('Joined shape:', joined.shape)

joined = joined[joined['ccs_group1'].notnull()]

print("The columns of joined are", joined.columns)

Pre-merge:
Diagnoses shape:  (651047, 5)
icds shape:  (15072, 9)
Post-merge:
Joined shape: (651047, 13)
The columns of joined are Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE', 'ccs_level1',
       'ccs_group1', 'ccs_level2', 'ccs_group2', 'ccs_level3', 'ccs_group3',
       'ccs_level4', 'ccs_group4'],
      dtype='object')


In [11]:
joined.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,ccs_level1,ccs_group1,ccs_level2,ccs_group2,ccs_level3,ccs_group3,ccs_level4,ccs_group4
0,3113,256,108811,1.0,53240,9.0,Diseases of the digestive system,9.1,Gastrointestinal hemorrhage [153.],9.10.1,Hemorrhage from gastrointestinal ulcer,NaN,NaN
1,3114,256,108811,2.0,41071,7.0,Diseases of the circulatory system,7.2,Diseases of the heart,7.2.3,Acute myocardial infarction [100.],NaN,NaN
2,3115,256,108811,3.0,53560,9.0,Diseases of the digestive system,9.4,Upper gastrointestinal disorders,9.4.3,Gastritis and duodenitis [140.],9.4.3.4,Duodenitis
3,3116,256,108811,4.0,40390,7.0,Diseases of the circulatory system,7.1,Hypertension,7.1.2,Hypertension with complications and secondary ...,7.1.2.1,Hypertensive heart and/or renal disease
4,3117,256,108811,5.0,5859,10.0,Diseases of the genitourinary system,10.1,Diseases of the urinary system,10.1.3,Chronic kidney disease [158.],NaN,NaN


In [12]:
#make column called category by transforming the 'ccs_group1' column and extracting the category

joined['category'] = joined['ccs_group1'].apply(extract_category)

In [13]:
joined.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,ccs_level1,ccs_group1,ccs_level2,ccs_group2,ccs_level3,ccs_group3,ccs_level4,ccs_group4,category
0,3113,256,108811,1.0,53240,9.0,Diseases of the digestive system,9.1,Gastrointestinal hemorrhage [153.],9.10.1,Hemorrhage from gastrointestinal ulcer,NaN,NaN,digestive
1,3114,256,108811,2.0,41071,7.0,Diseases of the circulatory system,7.2,Diseases of the heart,7.2.3,Acute myocardial infarction [100.],NaN,NaN,circulatory
2,3115,256,108811,3.0,53560,9.0,Diseases of the digestive system,9.4,Upper gastrointestinal disorders,9.4.3,Gastritis and duodenitis [140.],9.4.3.4,Duodenitis,digestive
3,3116,256,108811,4.0,40390,7.0,Diseases of the circulatory system,7.1,Hypertension,7.1.2,Hypertension with complications and secondary ...,7.1.2.1,Hypertensive heart and/or renal disease,circulatory
4,3117,256,108811,5.0,5859,10.0,Diseases of the genitourinary system,10.1,Diseases of the urinary system,10.1.3,Chronic kidney disease [158.],NaN,NaN,genitourinary


# Extract Patients who have sepsis 
- In the final csv that will be created, I will add a column that is a binary flag for sepsis or no sepsis (in addition to first_category). 
- This is in order to not lose the granularity of this information, when we only group diagnoses
- note: the people who will have sepsis=True, will be a portion of those whose 'first_category' is infectious|parasitic

In [14]:
#icd9 code for sepsis
icds[icds['icd9_code'].str.contains('99591')]

,icd9_code,ccs_level1,ccs_group1,ccs_level2,ccs_group2,ccs_level3,ccs_group3,ccs_level4,ccs_group4
450,99591,1,Infectious and parasitic diseases,1.1,Bacterial infection,1.1.2,Septicemia (except in labor) [2.],1.1.2.6,Unspecified septicemia


In [15]:
#extract those with sepsis
#this will be joined by subject_id and hadm_id at the end
patients_with_sepsis = joined[joined['ccs_group3'] == 'Septicemia (except in labor) [2.]']
patients_with_sepsis = patients_with_sepsis[['SUBJECT_ID', 'HADM_ID']]
patients_with_sepsis['sepsis'] = True
patients_with_sepsis = patients_with_sepsis.drop_duplicates().set_index(['SUBJECT_ID', 'HADM_ID'])

In [16]:
print('Patients with sepsis- shape: ',patients_with_sepsis.shape)
patients_with_sepsis.head()

Patients with sepsis- shape:  (8005, 1)


,,sepsis
SUBJECT_ID,HADM_ID,
1024,127666,True
4096,124383,True
72960,164620,True
74496,138943,True
9472,166597,True


In [17]:
patients_with_sepsis1 = patients_with_sepsis.reset_index()

In [18]:
patients_with_sepsis1.to_csv('../output_data/patients_with_sepsis.csv', index=False)

In [19]:
#more than one category per HADM_ID. so groupby HADM_ID to get the categories for each id
ids_to_categories = joined[['HADM_ID', 'category']].groupby('HADM_ID')['category'].apply(list).reset_index()

In [20]:
ids_to_categories['first_category'] = ids_to_categories['category'].apply(lambda x: x[0])

In [21]:
ids_to_categories.head()

,HADM_ID,category,first_category
0,100001,"[metabolic|immunity, nervous, genitourinary, d...",metabolic|immunity
1,100003,"[digestive, blood, infectious|parasitic, diges...",digestive
2,100006,"[respiratory, respiratory, respiratory, neopla...",respiratory
3,100007,"[digestive, circulatory, injury|poisoning, res...",digestive
4,100009,"[circulatory, injury|poisoning, circulatory, m...",circulatory


In [22]:
# one hot encode categories, and join back onto the main dataframe
hadm_item = pd.get_dummies(ids_to_categories['category'].apply(pd.Series).stack()).sum(level = 0)
hadm_item = ids_to_categories.join(hadm_item)

In [23]:
hadm_item.columns = [col.lower() for col in hadm_item.columns]
hadm_item.head()


,hadm_id,category,first_category,blood,circulatory,congenital,digestive,genitourinary,ill-defined,infectious|parasitic,...,mental,metabolic|immunity,musculoskeletal,neoplasms,nervous,perinatal,pregnancy complications,respiratory,skin,unclassified
0,100001,"[metabolic|immunity, nervous, genitourinary, d...",metabolic|immunity,0,2,0,2,2,1,0,...,0,5,1,0,2,0,0,0,1,0
1,100003,"[digestive, blood, infectious|parasitic, diges...",digestive,1,1,0,5,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,100006,"[respiratory, respiratory, respiratory, neopla...",respiratory,0,2,0,0,0,0,0,...,2,1,0,1,0,0,0,3,0,0
3,100007,"[digestive, circulatory, injury|poisoning, res...",digestive,0,2,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,100009,"[circulatory, injury|poisoning, circulatory, m...",circulatory,1,9,0,0,0,1,0,...,1,4,0,0,0,0,0,0,0,1


In [24]:

def apply_ones(df):
    for column in df.drop(['hadm_id', 'category', 'first_category'], axis=1).columns:
        df[column] = df[column].apply(lambda x: 1 if x > 0 else 0)
    return df
        
    
    
#ignore apply ones because when we go into clustering, we want to count the number of diagnoses as a column
#hadm_item = apply_ones(hadm_item)

In [25]:
print('Pre-merge: ')
print('Patient details: ', patient_details.shape)
print('hadm_iten: ', hadm_item.shape)
admissions_with_diagnosis = patient_details.merge(hadm_item, how='inner', on='hadm_id')
print('Post-merge with hadm_item:')
print('Admissions with diagnosis:', admissions_with_diagnosis.shape)

Pre-merge: 
Patient details:  (61532, 20)
hadm_iten:  (58929, 21)
Post-merge with hadm_item:
Admissions with diagnosis: (61525, 40)


In [26]:
admissions_with_diagnosis = admissions_with_diagnosis.join(icus, on='icustay_id')
print('Post-merge with icus:')
print('Admissions with diagnosis:', admissions_with_diagnosis.shape)

Post-merge with icus:
Admissions with diagnosis: (61525, 51)


## replace the intime column from patient_detials with the intime in icustays_trans_collapsed


In [27]:
#replace the intime column from patient_detials with the intime in intime_collapsed
#by joining on icustay_id

In [29]:
intime_collapsed.shape

(61532, 5)

In [30]:
print('Icu stay trans collapsed shape: ', intime_collapsed.shape)
intime_collapsed.head()

Icu stay trans collapsed shape:  (61532, 5)


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,INTIME_COLLAPSED,OUTTIME_COLLAPSED
0,1026,103873,215365,2007-04-28 05:19:34 UTC,2007-04-30 16:57:14 UTC
1,1970,144456,282130,2005-03-22 05:30:48 UTC,2005-03-29 17:17:07 UTC
2,5145,182528,289297,2006-02-08 00:05:43 UTC,2006-02-09 17:16:28 UTC
3,9967,171327,281112,2002-09-09 19:10:59 UTC,2002-09-18 16:56:11 UTC
4,15763,105221,255354,2004-04-05 10:24:39 UTC,2004-04-08 18:55:07 UTC


In [31]:
print('Admissions with diagnosis: Shape', admissions_with_diagnosis.shape)
admissions_with_diagnosis.head()

Admissions with diagnosis: Shape (61525, 51)


,subject_id,hadm_id,icustay_id,gender,dod,admittime,dischtime,los_hospital,age,ethnicity,...,SUBJECT_ID,HADM_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,2,163353,243653,M,NaN,2138-07-17 19:04:00,2138-07-21 15:48:00,4,0,ASIAN,...,2,163353,carevue,NICU,NICU,56,56,2138-07-17 21:20:07,2138-07-17 23:32:21,0.0918
1,3,145834,211552,M,2102-06-14 00:00:00,2101-10-20 19:08:00,2101-10-31 13:58:00,11,76,WHITE,...,3,145834,carevue,MICU,MICU,12,12,2101-10-20 19:10:11,2101-10-26 20:43:09,6.0646
2,4,185777,294638,F,NaN,2191-03-16 00:28:00,2191-03-23 18:41:00,7,48,WHITE,...,4,185777,carevue,MICU,MICU,52,52,2191-03-16 00:29:31,2191-03-17 16:46:31,1.6785
3,5,178980,214757,M,NaN,2103-02-02 04:31:00,2103-02-04 12:15:00,2,0,ASIAN,...,5,178980,carevue,NICU,NICU,56,56,2103-02-02 06:04:24,2103-02-02 08:06:00,0.0844
4,6,107064,228232,F,NaN,2175-05-30 07:15:00,2175-06-15 16:00:00,16,66,WHITE,...,6,107064,carevue,SICU,SICU,33,33,2175-05-30 21:30:54,2175-06-03 13:39:54,3.6729


In [32]:
joined = admissions_with_diagnosis.set_index('icustay_id').join(intime_collapsed.set_index('ICUSTAY_ID'), how='outer', lsuffix='awd_', rsuffix='ic_').reset_index()
joined = joined.rename(columns={'index': 'icustay_id'})


In [33]:
admissions_with_diagnosis = joined
print('Post-merge shape: ', joined.shape)

Post-merge shape:  (61532, 55)


In [34]:
admissions_with_diagnosis.head()

,icustay_id,subject_id,hadm_id,gender,dod,admittime,dischtime,los_hospital,age,ethnicity,...,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,SUBJECT_IDic_,HADM_IDic_,INTIME_COLLAPSED,OUTTIME_COLLAPSED
0,200001,55973.0,152234.0,F,2182-11-26 00:00:00,2181-11-18 16:04:00,2181-12-04 13:42:00,16.0,61.0,ASIAN - ASIAN INDIAN,...,MICU,23.0,23.0,2181-11-25 19:06:12,2181-11-28 20:59:25,3.0786,55973,152234,2011-09-04 19:06:12 UTC,2011-09-07 20:59:23 UTC
1,200003,27513.0,163557.0,M,2199-10-17 00:00:00,2199-08-02 17:02:00,2199-08-22 19:00:00,20.0,48.0,WHITE,...,SICU,57.0,57.0,2199-08-02 19:50:04,2199-08-08 17:09:18,5.8884,27513,163557,2007-07-13 19:50:04 UTC,2007-07-19 17:09:21 UTC
2,200006,10950.0,189514.0,M,2160-05-14 00:00:00,2159-09-03 11:27:00,2159-09-05 18:45:00,2.0,54.0,OTHER,...,MICU,52.0,52.0,2159-09-03 11:28:14,2159-09-04 19:08:10,1.3194,10950,189514,2004-11-22 11:28:14 UTC,2004-11-23 19:08:10 UTC
3,200007,20707.0,129310.0,M,NaN,2109-02-17 10:02:00,2109-02-20 15:47:00,3.0,44.0,WHITE,...,CCU,57.0,57.0,2109-02-17 10:03:37,2109-02-18 17:03:12,1.2914,20707,129310,2001-07-22 10:03:37 UTC,2001-07-23 17:03:13 UTC
4,200009,29904.0,129607.0,F,NaN,2189-11-30 10:45:00,2189-12-06 15:00:00,6.0,47.0,WHITE,...,CSRU,15.0,15.0,2189-11-30 10:34:32,2189-12-02 14:17:37,2.1549,29904,129607,2006-11-20 10:34:32 UTC,2006-11-22 14:17:35 UTC


In [35]:
admissions_with_diagnosis.shape

(61532, 55)

# Drop 'ethnicity' column because we have 'ethnicity_grouped' column

In [36]:
admissions_with_diagnosis = admissions_with_diagnosis.drop(['ethnicity'], axis=1)

# Check join

In [37]:
admissions_with_diagnosis.isnull().sum()

icustay_id                     0
subject_id                     7
hadm_id                        7
gender                         7
dod                        37348
admittime                      7
dischtime                      7
los_hospital                   7
age                            7
ethnicity_grouped              7
admission_type                 7
hospital_expire_flag           7
hospstay_seq                   7
first_hosp_stay                7
intime                         7
outtime                       17
los_icu                       17
icustay_seq                    7
first_icu_stay                 7
category                       7
first_category                 7
blood                          7
circulatory                    7
congenital                     7
digestive                      7
genitourinary                  7
ill-defined                    7
infectious|parasitic           7
injury|poisoning               7
mental                         7
metabolic|

# Binning age into groups
- Max age is 100.
- I bin ages into 5 groups.


In [38]:
def add_age_group(df):
    def group(x):
        if x >= 100:
            x = 99

        lower_bound = x // 20
        if lower_bound == 4:
            return '80-100'
        else:
            return f'{lower_bound * 20}-{(lower_bound + 1) * 20 - 1}'
    df['age_group'] = df['age'].apply(group)
    return df


In [39]:
admissions_with_diagnosis = add_age_group(admissions_with_diagnosis)
admissions_with_diagnosis.head()


,icustay_id,subject_id,hadm_id,gender,dod,admittime,dischtime,los_hospital,age,ethnicity_grouped,...,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,SUBJECT_IDic_,HADM_IDic_,INTIME_COLLAPSED,OUTTIME_COLLAPSED,age_group
0,200001,55973.0,152234.0,F,2182-11-26 00:00:00,2181-11-18 16:04:00,2181-12-04 13:42:00,16.0,61.0,asian,...,23.0,23.0,2181-11-25 19:06:12,2181-11-28 20:59:25,3.0786,55973,152234,2011-09-04 19:06:12 UTC,2011-09-07 20:59:23 UTC,60.0-79.0
1,200003,27513.0,163557.0,M,2199-10-17 00:00:00,2199-08-02 17:02:00,2199-08-22 19:00:00,20.0,48.0,white,...,57.0,57.0,2199-08-02 19:50:04,2199-08-08 17:09:18,5.8884,27513,163557,2007-07-13 19:50:04 UTC,2007-07-19 17:09:21 UTC,40.0-59.0
2,200006,10950.0,189514.0,M,2160-05-14 00:00:00,2159-09-03 11:27:00,2159-09-05 18:45:00,2.0,54.0,other,...,52.0,52.0,2159-09-03 11:28:14,2159-09-04 19:08:10,1.3194,10950,189514,2004-11-22 11:28:14 UTC,2004-11-23 19:08:10 UTC,40.0-59.0
3,200007,20707.0,129310.0,M,NaN,2109-02-17 10:02:00,2109-02-20 15:47:00,3.0,44.0,white,...,57.0,57.0,2109-02-17 10:03:37,2109-02-18 17:03:12,1.2914,20707,129310,2001-07-22 10:03:37 UTC,2001-07-23 17:03:13 UTC,40.0-59.0
4,200009,29904.0,129607.0,F,NaN,2189-11-30 10:45:00,2189-12-06 15:00:00,6.0,47.0,white,...,15.0,15.0,2189-11-30 10:34:32,2189-12-02 14:17:37,2.1549,29904,129607,2006-11-20 10:34:32 UTC,2006-11-22 14:17:35 UTC,40.0-59.0


# Adding binary column for death

In [40]:
admissions_with_diagnosis['dod_exists'] = admissions_with_diagnosis['dod'].notnull()

In [41]:
admissions_with_diagnosis.head()

,icustay_id,subject_id,hadm_id,gender,dod,admittime,dischtime,los_hospital,age,ethnicity_grouped,...,LAST_WARDID,INTIME,OUTTIME,LOS,SUBJECT_IDic_,HADM_IDic_,INTIME_COLLAPSED,OUTTIME_COLLAPSED,age_group,dod_exists
0,200001,55973.0,152234.0,F,2182-11-26 00:00:00,2181-11-18 16:04:00,2181-12-04 13:42:00,16.0,61.0,asian,...,23.0,2181-11-25 19:06:12,2181-11-28 20:59:25,3.0786,55973,152234,2011-09-04 19:06:12 UTC,2011-09-07 20:59:23 UTC,60.0-79.0,True
1,200003,27513.0,163557.0,M,2199-10-17 00:00:00,2199-08-02 17:02:00,2199-08-22 19:00:00,20.0,48.0,white,...,57.0,2199-08-02 19:50:04,2199-08-08 17:09:18,5.8884,27513,163557,2007-07-13 19:50:04 UTC,2007-07-19 17:09:21 UTC,40.0-59.0,True
2,200006,10950.0,189514.0,M,2160-05-14 00:00:00,2159-09-03 11:27:00,2159-09-05 18:45:00,2.0,54.0,other,...,52.0,2159-09-03 11:28:14,2159-09-04 19:08:10,1.3194,10950,189514,2004-11-22 11:28:14 UTC,2004-11-23 19:08:10 UTC,40.0-59.0,True
3,200007,20707.0,129310.0,M,NaN,2109-02-17 10:02:00,2109-02-20 15:47:00,3.0,44.0,white,...,57.0,2109-02-17 10:03:37,2109-02-18 17:03:12,1.2914,20707,129310,2001-07-22 10:03:37 UTC,2001-07-23 17:03:13 UTC,40.0-59.0,False
4,200009,29904.0,129607.0,F,NaN,2189-11-30 10:45:00,2189-12-06 15:00:00,6.0,47.0,white,...,15.0,2189-11-30 10:34:32,2189-12-02 14:17:37,2.1549,29904,129607,2006-11-20 10:34:32 UTC,2006-11-22 14:17:35 UTC,40.0-59.0,False


In [42]:
#checking the column for binary dod_exists is correct
print('length of df:',len(admissions_with_diagnosis))
print('no. of True dod_exists:',len(admissions_with_diagnosis[admissions_with_diagnosis['dod_exists']]))

print('length of df minus number of nulls in dod column:',len(admissions_with_diagnosis) - admissions_with_diagnosis.isnull().sum()['dod'])

length of df: 61532
no. of True dod_exists: 24184
length of df minus number of nulls in dod column: 24184


In [43]:
admissions_with_diagnosis.head()

,icustay_id,subject_id,hadm_id,gender,dod,admittime,dischtime,los_hospital,age,ethnicity_grouped,...,LAST_WARDID,INTIME,OUTTIME,LOS,SUBJECT_IDic_,HADM_IDic_,INTIME_COLLAPSED,OUTTIME_COLLAPSED,age_group,dod_exists
0,200001,55973.0,152234.0,F,2182-11-26 00:00:00,2181-11-18 16:04:00,2181-12-04 13:42:00,16.0,61.0,asian,...,23.0,2181-11-25 19:06:12,2181-11-28 20:59:25,3.0786,55973,152234,2011-09-04 19:06:12 UTC,2011-09-07 20:59:23 UTC,60.0-79.0,True
1,200003,27513.0,163557.0,M,2199-10-17 00:00:00,2199-08-02 17:02:00,2199-08-22 19:00:00,20.0,48.0,white,...,57.0,2199-08-02 19:50:04,2199-08-08 17:09:18,5.8884,27513,163557,2007-07-13 19:50:04 UTC,2007-07-19 17:09:21 UTC,40.0-59.0,True
2,200006,10950.0,189514.0,M,2160-05-14 00:00:00,2159-09-03 11:27:00,2159-09-05 18:45:00,2.0,54.0,other,...,52.0,2159-09-03 11:28:14,2159-09-04 19:08:10,1.3194,10950,189514,2004-11-22 11:28:14 UTC,2004-11-23 19:08:10 UTC,40.0-59.0,True
3,200007,20707.0,129310.0,M,NaN,2109-02-17 10:02:00,2109-02-20 15:47:00,3.0,44.0,white,...,57.0,2109-02-17 10:03:37,2109-02-18 17:03:12,1.2914,20707,129310,2001-07-22 10:03:37 UTC,2001-07-23 17:03:13 UTC,40.0-59.0,False
4,200009,29904.0,129607.0,F,NaN,2189-11-30 10:45:00,2189-12-06 15:00:00,6.0,47.0,white,...,15.0,2189-11-30 10:34:32,2189-12-02 14:17:37,2.1549,29904,129607,2006-11-20 10:34:32 UTC,2006-11-22 14:17:35 UTC,40.0-59.0,False


In [44]:
admissions_with_diagnosis.columns

Index(['icustay_id', 'subject_id', 'hadm_id', 'gender', 'dod', 'admittime',
       'dischtime', 'los_hospital', 'age', 'ethnicity_grouped',
       'admission_type', 'hospital_expire_flag', 'hospstay_seq',
       'first_hosp_stay', 'intime', 'outtime', 'los_icu', 'icustay_seq',
       'first_icu_stay', 'category', 'first_category', 'blood', 'circulatory',
       'congenital', 'digestive', 'genitourinary', 'ill-defined',
       'infectious|parasitic', 'injury|poisoning', 'mental',
       'metabolic|immunity', 'musculoskeletal', 'neoplasms', 'nervous',
       'perinatal', 'pregnancy complications', 'respiratory', 'skin',
       'unclassified', 'ROW_ID', 'SUBJECT_IDawd_', 'HADM_IDawd_', 'DBSOURCE',
       'FIRST_CAREUNIT', 'LAST_CAREUNIT', 'FIRST_WARDID', 'LAST_WARDID',
       'INTIME', 'OUTTIME', 'LOS', 'SUBJECT_IDic_', 'HADM_IDic_',
       'INTIME_COLLAPSED', 'OUTTIME_COLLAPSED', 'age_group', 'dod_exists'],
      dtype='object')

In [45]:
admissions_with_diagnosis = admissions_with_diagnosis.drop(['SUBJECT_IDic_', 'HADM_IDic_'], axis=1)

In [46]:
#admissions_with_diagnosis = admissions_with_diagnosis.drop(['INTIME_COLLAPSEDawd_', 'OUTTIME_COLLAPSEDawd_'], axis=1)

# Add Sepsis Info Back

In [48]:
admissions_with_diagnosis = admissions_with_diagnosis.join(patients_with_sepsis, on=['subject_id', 'hadm_id']).fillna({'sepsis': False})

admissions_with_diagnosis.head()

,icustay_id,subject_id,hadm_id,gender,dod,admittime,dischtime,los_hospital,age,ethnicity_grouped,...,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,INTIME_COLLAPSED,OUTTIME_COLLAPSED,age_group,dod_exists,sepsis
0,200001,55973.0,152234.0,F,2182-11-26 00:00:00,2181-11-18 16:04:00,2181-12-04 13:42:00,16.0,61.0,asian,...,23.0,23.0,2181-11-25 19:06:12,2181-11-28 20:59:25,3.0786,2011-09-04 19:06:12 UTC,2011-09-07 20:59:23 UTC,60.0-79.0,True,False
1,200003,27513.0,163557.0,M,2199-10-17 00:00:00,2199-08-02 17:02:00,2199-08-22 19:00:00,20.0,48.0,white,...,57.0,57.0,2199-08-02 19:50:04,2199-08-08 17:09:18,5.8884,2007-07-13 19:50:04 UTC,2007-07-19 17:09:21 UTC,40.0-59.0,True,True
2,200006,10950.0,189514.0,M,2160-05-14 00:00:00,2159-09-03 11:27:00,2159-09-05 18:45:00,2.0,54.0,other,...,52.0,52.0,2159-09-03 11:28:14,2159-09-04 19:08:10,1.3194,2004-11-22 11:28:14 UTC,2004-11-23 19:08:10 UTC,40.0-59.0,True,False
3,200007,20707.0,129310.0,M,NaN,2109-02-17 10:02:00,2109-02-20 15:47:00,3.0,44.0,white,...,57.0,57.0,2109-02-17 10:03:37,2109-02-18 17:03:12,1.2914,2001-07-22 10:03:37 UTC,2001-07-23 17:03:13 UTC,40.0-59.0,False,False
4,200009,29904.0,129607.0,F,NaN,2189-11-30 10:45:00,2189-12-06 15:00:00,6.0,47.0,white,...,15.0,15.0,2189-11-30 10:34:32,2189-12-02 14:17:37,2.1549,2006-11-20 10:34:32 UTC,2006-11-22 14:17:35 UTC,40.0-59.0,False,False


In [49]:
#investigation into sepsis
#corresponds with what we expect. 
#10% of patients who die, had sepsis
#28% of patients who die, did not have sepsis

#54% of patients who do not die, do not have sepsis

death_with_sepsis = admissions_with_diagnosis.groupby(['dod_exists', 'sepsis']).size()
(death_with_sepsis / sum(death_with_sepsis))

dod_exists  sepsis
False       False     0.551242
            True      0.055727
True        False     0.299828
            True      0.093204
dtype: float64

# Merge diagnosis categories
- 'perinatal' = ('congenital', 'perinatal', 'pregnancy complications')
- 'circulatory' = ('circulatory', 'blood')
- 'other' = ('ill-defined', 'unclassified',  'genitourinary', 'skin')

In [50]:
#list of unique diagnosis before transformation
print(len(admissions_with_diagnosis['first_category'].unique()))
list(admissions_with_diagnosis['first_category'].unique())


19


['skin',
 'infectious|parasitic',
 'injury|poisoning',
 'circulatory',
 'digestive',
 'respiratory',
 'perinatal',
 'nervous',
 'neoplasms',
 'mental',
 'metabolic|immunity',
 'musculoskeletal',
 'congenital',
 'genitourinary',
 'ill-defined',
 'blood',
 'unclassified',
 'pregnancy complications',
 nan]

In [51]:
def merge_first_categories(df):
    #perinatal' = ('congenital', 'perinatal', 'pregnancy complications')
    df.replace(to_replace =["congenital", 'pregnancy complications'], value ="perinatal", inplace=True) 
    
    #circulatory' = ('circulatory', 'blood')
    df.replace(to_replace ="blood", value ="circulatory", inplace=True) 
    
    #other' = ('ill-defined', 'unclassified',  'genitourinary', 'skin')
    df.replace(to_replace =['ill-defined', 'unclassified', 'genitourinary', 'skin'], value ='other', inplace=True) 
    return df

admissions_with_diagnosis = merge_first_categories(admissions_with_diagnosis)



In [52]:
#list of unique diagnosis after transformation
print(len(admissions_with_diagnosis['first_category'].unique()))
list(admissions_with_diagnosis['first_category'].unique())


13


['other',
 'infectious|parasitic',
 'injury|poisoning',
 'circulatory',
 'digestive',
 'respiratory',
 'perinatal',
 'nervous',
 'neoplasms',
 'mental',
 'metabolic|immunity',
 'musculoskeletal',
 nan]

# Add Diagnosis counts column

In [53]:
def add_diagnosis_counts(df):
    diagnoses_df = df[['blood', 'circulatory',
       'congenital', 'digestive', 'genitourinary', 'ill-defined',
       'infectious|parasitic', 'injury|poisoning', 'mental',
       'metabolic|immunity', 'musculoskeletal', 'neoplasms', 'nervous',
       'perinatal', 'pregnancy complications', 'respiratory', 'skin',
       'unclassified']]
    
    df['diagnoses_count'] = diagnoses_df.sum(axis=1)
    return df

In [54]:
admissions_with_diagnosis = add_diagnosis_counts(admissions_with_diagnosis)

In [55]:
len(admissions_with_diagnosis.columns)

56

# Write to csv

In [56]:
admissions_with_diagnosis.head()

,icustay_id,subject_id,hadm_id,gender,dod,admittime,dischtime,los_hospital,age,ethnicity_grouped,...,LAST_WARDID,INTIME,OUTTIME,LOS,INTIME_COLLAPSED,OUTTIME_COLLAPSED,age_group,dod_exists,sepsis,diagnoses_count
0,200001,55973.0,152234.0,F,2182-11-26 00:00:00,2181-11-18 16:04:00,2181-12-04 13:42:00,16.0,61.0,asian,...,23.0,2181-11-25 19:06:12,2181-11-28 20:59:25,3.0786,2011-09-04 19:06:12 UTC,2011-09-07 20:59:23 UTC,60.0-79.0,True,False,20.0
1,200003,27513.0,163557.0,M,2199-10-17 00:00:00,2199-08-02 17:02:00,2199-08-22 19:00:00,20.0,48.0,white,...,57.0,2199-08-02 19:50:04,2199-08-08 17:09:18,5.8884,2007-07-13 19:50:04 UTC,2007-07-19 17:09:21 UTC,40.0-59.0,True,True,6.0
2,200006,10950.0,189514.0,M,2160-05-14 00:00:00,2159-09-03 11:27:00,2159-09-05 18:45:00,2.0,54.0,other,...,52.0,2159-09-03 11:28:14,2159-09-04 19:08:10,1.3194,2004-11-22 11:28:14 UTC,2004-11-23 19:08:10 UTC,40.0-59.0,True,False,8.0
3,200007,20707.0,129310.0,M,NaN,2109-02-17 10:02:00,2109-02-20 15:47:00,3.0,44.0,white,...,57.0,2109-02-17 10:03:37,2109-02-18 17:03:12,1.2914,2001-07-22 10:03:37 UTC,2001-07-23 17:03:13 UTC,40.0-59.0,False,False,7.0
4,200009,29904.0,129607.0,F,NaN,2189-11-30 10:45:00,2189-12-06 15:00:00,6.0,47.0,white,...,15.0,2189-11-30 10:34:32,2189-12-02 14:17:37,2.1549,2006-11-20 10:34:32 UTC,2006-11-22 14:17:35 UTC,40.0-59.0,False,False,6.0


In [57]:
print('length of csv:',len(admissions_with_diagnosis))

length of csv: 61532


In [10]:

# Chloe's address
#admissions_with_diagnosis.to_csv('/Users/chloemaine/Documents/Chloe/BGSE/masters_project/processed_data/admissions_with_diagnosis_assigned_icu.csv', index=False)

# Add on Ideal ICU column from patient_details_ideal_icu.csv

In [58]:
len(admissions_with_diagnosis)

61532

In [59]:
# Chloe's directory
#os.chdir('/Users/chloemaine/Documents/Chloe/BGSE/masters_project/processed_data')
patient_details_idealicu = pd.read_csv('../input_data/patient_details_idealicu.csv')
patient_details_idealicu = patient_details_idealicu[['subject_id', 'hadm_id', 'icustay_id', 'ideal_ICU']]

In [60]:
len(patient_details_idealicu)

61532

In [61]:
admissions_with_diagnosis = admissions_with_diagnosis.set_index(['subject_id', 'hadm_id', 'icustay_id']).join(patient_details_idealicu.set_index(['subject_id', 'hadm_id', 'icustay_id']))


In [62]:
len(admissions_with_diagnosis)

61532

In [64]:
admissions_with_diagnosis.head()

,,,gender,dod,admittime,dischtime,los_hospital,age,ethnicity_grouped,admission_type,hospital_expire_flag,hospstay_seq,...,INTIME,OUTTIME,LOS,INTIME_COLLAPSED,OUTTIME_COLLAPSED,age_group,dod_exists,sepsis,diagnoses_count,ideal_ICU
subject_id,hadm_id,icustay_id,,,,,,,,,,,,,,,,,,,,,
55973.0,152234.0,200001,F,2182-11-26 00:00:00,2181-11-18 16:04:00,2181-12-04 13:42:00,16.0,61.0,asian,EMERGENCY,0.0,6.0,...,2181-11-25 19:06:12,2181-11-28 20:59:25,3.0786,2011-09-04 19:06:12 UTC,2011-09-07 20:59:23 UTC,60.0-79.0,True,False,20.0,MICU
27513.0,163557.0,200003,M,2199-10-17 00:00:00,2199-08-02 17:02:00,2199-08-22 19:00:00,20.0,48.0,white,EMERGENCY,0.0,1.0,...,2199-08-02 19:50:04,2199-08-08 17:09:18,5.8884,2007-07-13 19:50:04 UTC,2007-07-19 17:09:21 UTC,40.0-59.0,True,True,6.0,SICU
10950.0,189514.0,200006,M,2160-05-14 00:00:00,2159-09-03 11:27:00,2159-09-05 18:45:00,2.0,54.0,other,EMERGENCY,0.0,3.0,...,2159-09-03 11:28:14,2159-09-04 19:08:10,1.3194,2004-11-22 11:28:14 UTC,2004-11-23 19:08:10 UTC,40.0-59.0,True,False,8.0,MICU
20707.0,129310.0,200007,M,NaN,2109-02-17 10:02:00,2109-02-20 15:47:00,3.0,44.0,white,EMERGENCY,0.0,1.0,...,2109-02-17 10:03:37,2109-02-18 17:03:12,1.2914,2001-07-22 10:03:37 UTC,2001-07-23 17:03:13 UTC,40.0-59.0,False,False,7.0,CCU
29904.0,129607.0,200009,F,NaN,2189-11-30 10:45:00,2189-12-06 15:00:00,6.0,47.0,white,ELECTIVE,0.0,1.0,...,2189-11-30 10:34:32,2189-12-02 14:17:37,2.1549,2006-11-20 10:34:32 UTC,2006-11-22 14:17:35 UTC,40.0-59.0,False,False,6.0,CSRU


In [65]:
admissions_with_diagnosis = admissions_with_diagnosis.drop(['ROW_ID', 
                                                            'HADM_IDawd_', 
                                                            'SUBJECT_IDawd_',
                                                            'DBSOURCE', 
                                                            'LAST_CAREUNIT', 
                                                            'FIRST_WARDID', 
                                                            'LAST_WARDID',
                                                            'OUTTIME', 
                                                            'LOS',  
                                                            'age', 
                                                            'intime',
                                                            'outtime'], axis=1)

In [66]:
admissions_with_diagnosis.head()


,,,gender,dod,admittime,dischtime,los_hospital,ethnicity_grouped,admission_type,hospital_expire_flag,hospstay_seq,first_hosp_stay,...,unclassified,FIRST_CAREUNIT,INTIME,INTIME_COLLAPSED,OUTTIME_COLLAPSED,age_group,dod_exists,sepsis,diagnoses_count,ideal_ICU
subject_id,hadm_id,icustay_id,,,,,,,,,,,,,,,,,,,,,
55973.0,152234.0,200001,F,2182-11-26 00:00:00,2181-11-18 16:04:00,2181-12-04 13:42:00,16.0,asian,EMERGENCY,0.0,6.0,False,...,2.0,MICU,2181-11-25 19:06:12,2011-09-04 19:06:12 UTC,2011-09-07 20:59:23 UTC,60.0-79.0,True,False,20.0,MICU
27513.0,163557.0,200003,M,2199-10-17 00:00:00,2199-08-02 17:02:00,2199-08-22 19:00:00,20.0,white,EMERGENCY,0.0,1.0,True,...,0.0,SICU,2199-08-02 19:50:04,2007-07-13 19:50:04 UTC,2007-07-19 17:09:21 UTC,40.0-59.0,True,True,6.0,SICU
10950.0,189514.0,200006,M,2160-05-14 00:00:00,2159-09-03 11:27:00,2159-09-05 18:45:00,2.0,other,EMERGENCY,0.0,3.0,False,...,1.0,MICU,2159-09-03 11:28:14,2004-11-22 11:28:14 UTC,2004-11-23 19:08:10 UTC,40.0-59.0,True,False,8.0,MICU
20707.0,129310.0,200007,M,NaN,2109-02-17 10:02:00,2109-02-20 15:47:00,3.0,white,EMERGENCY,0.0,1.0,True,...,1.0,CCU,2109-02-17 10:03:37,2001-07-22 10:03:37 UTC,2001-07-23 17:03:13 UTC,40.0-59.0,False,False,7.0,CCU
29904.0,129607.0,200009,F,NaN,2189-11-30 10:45:00,2189-12-06 15:00:00,6.0,white,ELECTIVE,0.0,1.0,True,...,1.0,CSRU,2189-11-30 10:34:32,2006-11-20 10:34:32 UTC,2006-11-22 14:17:35 UTC,40.0-59.0,False,False,6.0,CSRU


# Drop rows where intime_trans_collapsed is null
(10 rows to drop)

In [67]:
admissions_with_diagnosis.reset_index(inplace=True)

In [68]:
admissions_with_diagnosis.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'gender', 'dod', 'admittime',
       'dischtime', 'los_hospital', 'ethnicity_grouped', 'admission_type',
       'hospital_expire_flag', 'hospstay_seq', 'first_hosp_stay', 'los_icu',
       'icustay_seq', 'first_icu_stay', 'category', 'first_category', 'blood',
       'circulatory', 'congenital', 'digestive', 'genitourinary',
       'ill-defined', 'infectious|parasitic', 'injury|poisoning', 'mental',
       'metabolic|immunity', 'musculoskeletal', 'neoplasms', 'nervous',
       'perinatal', 'pregnancy complications', 'respiratory', 'skin',
       'unclassified', 'FIRST_CAREUNIT', 'INTIME', 'INTIME_COLLAPSED',
       'OUTTIME_COLLAPSED', 'age_group', 'dod_exists', 'sepsis',
       'diagnoses_count', 'ideal_ICU'],
      dtype='object')

In [69]:
drop_indexes = admissions_with_diagnosis[admissions_with_diagnosis['INTIME_COLLAPSED'].isnull()].index
len(drop_indexes)

0

In [70]:
admissions_with_diagnosis = admissions_with_diagnosis.drop(drop_indexes)

In [71]:
len(admissions_with_diagnosis)

61532

# output csv 
will go into data_split.ipynb


In [72]:
#this csv will go into data_split.ipynb
admissions_with_diagnosis.to_csv('../output_data/cleaned_data_ideal_and_assigned.csv', index=False)
